<a href="https://colab.research.google.com/github/bgrantham05/ACMAITransformersSpring2025/blob/main/ACMai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip uninstall -y torch torchvision
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118


!pip install numpy pandas tabulate tokenizers datasets transformers peft accelerate xformers bitsandbytes jupyter jupytext seaborn matplotlib tqdm torchvision

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 99.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 121.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB ? eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 894.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

^C


In [1]:
import math
import torch
from pathlib import Path
from typing import Union, Dict
import pandas as pd
import numpy as np

import pathlib
from typing import List, Dict, Tuple, Optional

import torch
import torch.nn as nn

# Basic Model Imports
from transformers import RobertaTokenizer, RobertaTokenizerFast, RobertaModel

# Import Attentions to adjust
from torch.nn import MultiheadAttention
from transformers.models.bert.modeling_bert import BertSelfAttention
from transformers.models.roberta.modeling_roberta import RobertaSelfAttention

# Import Own Modules
# import lora_utils

from torch.nn import functional as F

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jithinanievarghese/drugs-side-effects-and-medical-condition")

print("Path to dataset files:", path)

100%|██████████| 1.17M/1.17M [00:00<00:00, 2.29MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/jithinanievarghese/drugs-side-effects-and-medical-condition/versions/1


In [3]:
# Convert to Path object
data_path = Path(path)

# Find the CSV file inside the folder
csv_files = list(data_path.rglob("*.csv"))
if not csv_files:
    raise FileNotFoundError("No CSV files found in the dataset folder.")

# Load the CSV file (choose the first one found)
df = pd.read_csv(csv_files[0])
print("Original columns:", df.columns.tolist())

# Choose only two columns to keep
# Replace these with the actual column names from the dataset
columns_to_keep = ["drug_name", "medical_condition"]  # <-- update these to match the dataset
clean_df = df[columns_to_keep].dropna()

print("Cleaned DataFrame:")
print(clean_df.head())

Original columns: ['drug_name', 'medical_condition', 'side_effects', 'generic_name', 'drug_classes', 'brand_names', 'activity', 'rx_otc', 'pregnancy_category', 'csa', 'alcohol', 'related_drugs', 'medical_condition_description', 'rating', 'no_of_reviews', 'drug_link', 'medical_condition_url']
Cleaned DataFrame:
        drug_name medical_condition
0     doxycycline              Acne
1  spironolactone              Acne
2     minocycline              Acne
3        Accutane              Acne
4     clindamycin              Acne


In [11]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Ensure padding is valid
model = GPT2LMHeadModel.from_pretrained("gpt2")


def tokenize_fn(example):
    # Format the prompt and the target
    input_text = f"Condition: {example['medical_condition']}\nDrug:"
    target_text = f"{example['drug_name'].strip()}"

    # Tokenize prompt and target
    input_enc = tokenizer(input_text, truncation=True, padding=False)
    target_enc = tokenizer(target_text, truncation=True, padding=False)

    input_ids = input_enc["input_ids"] + target_enc["input_ids"]
    labels = [-100] * len(input_enc["input_ids"]) + target_enc["input_ids"]

    # Set max_length to fit prompt + short target (1 word)
    max_length = 64
    pad_len = max_length - len(input_ids)

    if pad_len > 0:
        input_ids += [tokenizer.pad_token_id] * pad_len
        labels += [-100] * pad_len
    else:
        input_ids = input_ids[:max_length]
        labels = labels[:max_length]

    assert len(input_ids) == max_length
    assert len(labels) == max_length

    return {"input_ids": input_ids, "labels": labels}


In [12]:
from datasets import Dataset

dataset = Dataset.from_pandas(clean_df)
tokenized_dataset = dataset.map(tokenize_fn, remove_columns=dataset.column_names)

Map:   0%|          | 0/2931 [00:00<?, ? examples/s]

In [13]:
from transformers import AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType

model = AutoModelForCausalLM.from_pretrained("gpt2")

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["c_attn", "c_proj"],  # adjust depending on the model
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [14]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    num_train_epochs=5,
    learning_rate=1e-4,
    logging_dir="./logs",
    save_strategy="epoch",
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()

<ipython-input-14-4c068d066cfa>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
500,5.370000
1000,4.947400
1500,4.899800


TrainOutput(global_step=1835, training_loss=5.023544203423024, metrics={'train_runtime': 146.7143, 'train_samples_per_second': 99.888, 'train_steps_per_second': 12.507, 'total_flos': 483218304860160.0, 'train_loss': 5.023544203423024, 'epoch': 5.0})

In [15]:
model.save_pretrained("lora-med-gen")
tokenizer.save_pretrained("lora-med-gen")

('lora-med-gen/tokenizer_config.json',
 'lora-med-gen/special_tokens_map.json',
 'lora-med-gen/vocab.json',
 'lora-med-gen/merges.txt',
 'lora-med-gen/added_tokens.json')

In [16]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Load base and LoRA-augmented model
base_model = AutoModelForCausalLM.from_pretrained("gpt2")
model = PeftModel.from_pretrained(base_model, "lora-med-gen")

tokenizer = AutoTokenizer.from_pretrained("lora-med-gen")

In [17]:
import re

def generate_drug_name(condition: str) -> str:
    prompt = f"Condition: {condition}\nDrug:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=32,
        num_return_sequences=1,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.9,
        pad_token_id=tokenizer.eos_token_id
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only first 1–2 valid words
    stripped = generated_text[len(prompt):].strip() if generated_text.startswith(prompt) else generated_text.strip()
    words = re.findall(r"\b[A-Za-z][a-zA-Z\-]{1,}\b", stripped)

    if not words:
        return "[Unknown]"
    return " ".join(words[:2])  # return 1 or 2 words max
while True:
    user_input = input("Enter a medical condition (or type 'exit' to quit): ").strip()
    if user_input.lower() == "exit":
        break
    drug = generate_drug_name(user_input)
    print(f"AI-generated drug: {drug}\n")


Enter a medical condition (or type 'exit' to quit): acne
AI-generated drug: Mifetil Month

Enter a medical condition (or type 'exit' to quit): hyperhydrosis
AI-generated drug: Nilikovas TNF-N

Enter a medical condition (or type 'exit' to quit): influenza
AI-generated drug: Tacavac-A X-Maxim-Therapeutic

Enter a medical condition (or type 'exit' to quit): kidney failure
AI-generated drug: chlorpheniramine phenylalanine

Enter a medical condition (or type 'exit' to quit): brain failure
AI-generated drug: Loxivac Med

Enter a medical condition (or type 'exit' to quit): death
AI-generated drug: Ishmao Cold-Fingers

Enter a medical condition (or type 'exit' to quit): lupus
AI-generated drug: acetaminophen ibuprofen

Enter a medical condition (or type 'exit' to quit): fever
AI-generated drug: Dia-Gueca Medivac

Enter a medical condition (or type 'exit' to quit): encephalitis
AI-generated drug: Etrexa-A Ease

Enter a medical condition (or type 'exit' to quit): food poisoning
AI-generated drug